In [68]:
import pandas as pd
import numpy as np

Setting N (Number of validators in era) fixed for now

In [1]:
initial_N = 297

Loading of data from era number 165 and session 1031

In [7]:
nominators = pd.read_csv("data/polkadot_nominators_session_1031.csv")
validators = pd.read_csv("data/polkadot_validators_era_165.csv")

In [46]:
nominators

,era,session,block_number,stash_address,controller_address,bonded_amount,num_targets,targets
0,165,1031,2465718,16Ak2D2ZApaR6sFKkawmC8ea93ASD4aB54eTMvEkDjCFZLjp,16Ak2D2ZApaR6sFKkawmC8ea93ASD4aB54eTMvEkDjCFZLjp,39498643450307,1,1hJdgnAPSjfuHZFHzcorPnFvekSHihK9jdNPWHXgeuL7zaJ
1,165,1031,2465718,14FzsWSxgWMmXwafTiGf1YdXZgEoAY7dEv2Qr9h1sCHaWFt2,14FzsWSxgWMmXwafTiGf1YdXZgEoAY7dEv2Qr9h1sCHaWFt2,1084592814654,16,1zugcacYFxX3HveFpJVUShjfb3KyaomfVqMTFoxYuUWCdD...
2,165,1031,2465718,15LL8mQJQ7wsxrjxMyBxjphkNxciwirC6cQ5AprmmEt8KGNf,15wtWHaQemsJL4uPKGKCRwnpaGUK66VdLjd8wJKWcpoEtetf,36219110590042,16,1zugcabYjgfQdMLC3cAzQ8tJZMo45tMnGpivpAzpxB4CZy...
3,165,1031,2465718,11888uchjZxbyc7RRstxZ8qepsskRjJhJatrxryBTdHuWfK,11888uchjZxbyc7RRstxZ8qepsskRjJhJatrxryBTdHuWfK,2352067003888369,5,14cxMDpBNLsNEXWyCzked3zghzaYWXwoqGT4h12GqQXdVh...
4,165,1031,2465718,1ZQpD1knimbf4RLUyoipThPdhvEXJHud2sFDSe3cWPVN3q4,1ZQpD1knimbf4RLUyoipThPdhvEXJHud2sFDSe3cWPVN3q4,3894263543601,16,14SRtmkZPGzaV1bKivd7T6xw15YRW53gjE5F2LVbiwChqK...
...,...,...,...,...,...,...,...,...
8198,165,1031,2465718,16XMi2A7NvUuwt632GH7awa7P5MYJQrteJJ3iB13FCqeXPok,16XMi2A7NvUuwt632GH7awa7P5MYJQrteJJ3iB13FCqeXPok,2780000000000,10,15awXd39ZEttFj5tyM6Lkq5ReefCNcrG8f2agSgn2ZCjoJ...
8199,165,1031,2465718,16A5SmtCw1sTDr6zXdJ9hShqUX1XbYKXG1kk8Qv2BPhTsDNC,16A5SmtCw1sTDr6zXdJ9hShqUX1XbYKXG1kk8Qv2BPhTsDNC,245783340884,16,12vCVEpCz81rvS23PEJQ6KBDbPWgvyVX5Q1PrgQbVvLc4i...
8200,165,1031,2465718,14N6uWVnUDZQfNf66HJsSFPLmJHm1p19Y44CBmVoBi9DDbRD,14N6uWVnUDZQfNf66HJsSFPLmJHm1p19Y44CBmVoBi9DDbRD,14529640,4,12LKeuFyyjC94iXpHftt3UVu567ji5WyKE6MvDBDWVJUuu...
8201,165,1031,2465718,13gfodUESWspVuk5xHdkC8uCJi3EAvTtjiaRCUsN9ktwhpKR,13gfodUESWspVuk5xHdkC8uCJi3EAvTtjiaRCUsN9ktwhpKR,120000000000000,5,15DLJZ4ceN58vEgDiQjK8JsSJuLNBqhUnQ6QCY1QNSjrQn...


In [9]:
validators

,era,session,block_number,name,stash_address,controller_address,commission_percent,self_stake,total_stake,num_stakers,voters,era_points
0,165,1031,2465718,Wei,14ShUZUYUR35RBZW6uVVt1zXDxmSQddkeDdXf1JkMA6P721N,16knCdWHWokjeciJuassAub21PHi3YTt94um4DjRJSRtF7sh,1.0,2331305558010386,30622799978313560,127,1831,1260
1,165,1031,2465718,NaN,1FCu68ZwBHNzZLcGa92eHwvR61hk3MpjrhiqN96xF9vWS1Q,13gAjcC56upxCdaNS69jB7rvGDzimcztTm9Jxhfgma8w9jor,100.0,99360406100,37263464975913580,2,27,1300
2,165,1031,2465718,Zug Cap / 19,1zugcag7cJVBtVRnFxv5Qftn7xKAnR6YJ9x4x3XLgGgmNnS,13eeBGJPSPQjXGfBmd2rsek8gKU1ivz7eZReiVRwwES1QzuX,0.7,100000000000,30549484370550930,106,1203,1260
3,165,1031,2465718,Polkadot.pro - Realgar,1REAJ1k691g5Eqqg9gL7vvZCBG7FCCZ8zgQkZWd4va5ESih,1ZMbuCR3QiatxRsQdNnJYgydn3CWV4PELcTzpH4TNoNjxno,1.0,5342981819844241,34321621635570840,128,1329,1120
4,165,1031,2465718,Able Wanderer / 🐁 Mouse-6,135wKSGZpQcTrhmFju9VeKC68gMBMsCYpSNtDyoqj3BVSZ1R,135wKSGZpQcTrhmFju9VeKC68gMBMsCYpSNtDyoqj3BVSZ1R,100.0,10000000000,26135922035705640,4,9,1180
...,...,...,...,...,...,...,...,...,...,...,...,...
223,165,1031,2465718,NaN,14yVcx8AtnmaZMCZWVp8AKDgnAYdjeQjAbPL1HbvPxr74Fsf,16H1vG9t6EzS64r4YgLHBEXwKkVG23gE8t84DzXTtUi1ucxv,100.0,99360406100,24840579921059186,1,1,1200
224,165,1031,2465718,P2P.ORG / 18,16Sud9b5uUfUi1HXdfwb3drbYBZBLPVvdKuZhwxz2n7HR12M,16Sud9b5uUfUi1HXdfwb3drbYBZBLPVvdKuZhwxz2n7HR12M,3.0,100000533506,30626119242918828,50,283,1280
225,165,1031,2465718,🔒stateless_money🔒,12713bbq45c66CN9AD7yusSXWE1kY91DcMpjVcB2rXqZKy2w,1f2K1YccCZEfbcGrHshQWzrGdXp6bxFgMahGrKYZm9HPuy1,1.0,2000590510261,31150322561480132,15,95,1220
226,165,1031,2465718,NaN,15rQD4ykz4uRD7YGdTpMT8UmgkNDJRKckZbAocALndBVzCjf,15rQD4ykz4uRD7YGdTpMT8UmgkNDJRKckZbAocALndBVzCjf,100.0,10000000000,24620024373998627,3,7,1020


In [26]:
target = nominators["targets"][0]
validators[validators["stash_address"] == target]

,era,session,block_number,name,stash_address,controller_address,commission_percent,self_stake,total_stake,num_stakers,voters,era_points
97,165,1031,2465718,Listen,1hJdgnAPSjfuHZFHzcorPnFvekSHihK9jdNPWHXgeuL7zaJ,136JdPA8uidFAUPJKarwpBsPqGkkRqLTYgNBC7vuwyAnLd6a,5.0,1799881215889431,30614677019378986,171,550,1580


Creation of binary table similar to one in presentation, so showing which validators nominator selected and his total money

In [67]:
binary_matrix = nominators["targets"].str.get_dummies(sep=',')
binary_matrix.index = nominators["stash_address"]
binary_matrix["amount"] = nominators.set_index("stash_address")["bonded_amount"]
binary_matrix

,111111111111111111111111111111111HC1,1124RsfEgJEZvEq4HbtGFcpqoxnqSy79EjNZY9tzPct3AB6o,112Ji3ASaU21FGa4ymrjvyZrDv8LbW9tF5NLUksdLdJkjY7E,1155dDdp1X4F3rh35hAMoK8r4iDVdzprtRpVrScP35YPC2b,11BgR7fH8Sq6CcGcXxZrhyrBM2PUpDmhnGZpxPGvVGXEiPT,11MJU5Q1rQh5BKuuECePhSAutv3WEVx6f2x9eZk9HXkCC1e,11gqpAyU17G9EFW5n5MNngh824F3Y2as72V2rgc7Wq5JVRd,11uMPbeaEDJhUxzU4ZfWW9VQEsryP9XqFcNRfPdYda6aFWJ,121GBc2ZK53Uoc6JovJjU7zARH8fys5efyUfHKT9vABpqajx,121ZiNk5DKVKUuYQtuNcHC25AD2K8bSNaaFn7qSa4JrSwYBR,...,1zugcauBVvcNa6tpW3j6WaGRL1Wvm9oSimK2eP3VNqNLw1V,1zugcavJYzi2KErZy9CMbLANhfrFwMESgPz9q29eUCR5gTW,1zugcavYA9yCuYwiEYeMHNJm9gXznYjNfXQjZsZukF1Mpow,1zugcawsx74AgoC4wz2dMEVFVDNo7rVuTRjZMnfNp9T49po,1zugcaxRrQDr7ktb6SpjVMkywys2ysoZWBfUfs9CDPwFVHC,1zugcaxcGLmRb6wFpftx99sYRSQihqq6KLTThimEkYsaSoq,1zugcazy9vaQJkMX79DYwFdvTYGmAbjPQrYuyUNHoFnXKtA,1zugcdarkTpYvG69VFJvxCdMVovSoWPieafJgLCnWCsYCzS,1zugchKTRDgvvfeGGSf5KpVWMx17Atf956FJYaFrmxdS4T7,amount
stash_address,,,,,,,,,,,,,,,,,,,,,
16Ak2D2ZApaR6sFKkawmC8ea93ASD4aB54eTMvEkDjCFZLjp,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,39498643450307
14FzsWSxgWMmXwafTiGf1YdXZgEoAY7dEv2Qr9h1sCHaWFt2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1084592814654
15LL8mQJQ7wsxrjxMyBxjphkNxciwirC6cQ5AprmmEt8KGNf,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,36219110590042
11888uchjZxbyc7RRstxZ8qepsskRjJhJatrxryBTdHuWfK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2352067003888369
1ZQpD1knimbf4RLUyoipThPdhvEXJHud2sFDSe3cWPVN3q4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3894263543601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16XMi2A7NvUuwt632GH7awa7P5MYJQrteJJ3iB13FCqeXPok,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2780000000000
16A5SmtCw1sTDr6zXdJ9hShqUX1XbYKXG1kk8Qv2BPhTsDNC,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,245783340884
14N6uWVnUDZQfNf66HJsSFPLmJHm1p19Y44CBmVoBi9DDbRD,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,14529640


Creation of table as above, but with number of how many specific validators nominator selected instead of information if validator is present in targets of nominator.

In [69]:
unique_validators = pd.Series(nominators["targets"].str.split(",").sum()).unique()
count_matrix_np = np.zeros((len(nominators), len(unique_validators)), dtype=int)
validator_to_idx = {validator: i for i, validator in enumerate(unique_validators)}

for i, targets in enumerate(nominators['targets']):
    for validator in targets.split(','):
        count_matrix_np[i, validator_to_idx[validator]] += 1

count_matrix = pd.DataFrame(count_matrix_np, index=nominators['stash_address'], columns=unique_validators)
count_matrix['amount'] = nominators.set_index('stash_address')['bonded_amount']
count_matrix

,1hJdgnAPSjfuHZFHzcorPnFvekSHihK9jdNPWHXgeuL7zaJ,1zugcacYFxX3HveFpJVUShjfb3KyaomfVqMTFoxYuUWCdD8,1zugcaebzKgKLebGSQvtxpmPGCZLFoEVu6AfqwD7W5ZKQZt,1zugcakrhr3ZR7q7B8WKuaZY5BjZAU43m79xEyhNQwLTFjb,1zugcarJnZ4ft2PiJoGg6DgmZjnKNBrcKTFrAzhGPCX6bJ5,1zugcaxRrQDr7ktb6SpjVMkywys2ysoZWBfUfs9CDPwFVHC,1zugcabTuN7rs1bFYb33gRemtg67i4Mvp1twW85nQKiwhwQ,1zugcaj4mBMu7EULN4rafT5UTfBjbvqaoypZyxWa3io6qJS,14ShUZUYUR35RBZW6uVVt1zXDxmSQddkeDdXf1JkMA6P721N,16GDRhRYxk42paoK6TfHAqWej8PdDDUwdDazjv4bAn4KGNeb,...,15YRqDv5mYuVoeFcW7Y2NWgB5Ut6bc9C8y35yAC7aT6P3YzD,15BTJ9g5Rqm9PxHXh6sqMM5QVDXpwtQCTHp5MvbHbxnkvivy,13iXZVPpXuRnrSmKsqM51NtR8nqD5arfovegQT75r3Pp1Hpe,1zugca4bLxr1BB6gngcKaZmV8u76tmAJagg6FhePZjNJmam,13b7JdEzrXvGbkACdrbCXayeMnbcYT5NMK6paWdmM9328xsw,15r3BYwQ7ucMirgWuaZt5L1GxJt7JxbDtUWq8S8381B8LqZf,135zW8niMZUNGZQ7e8V1wSXzyaicfq1rAgpXugzxQkrHr1Mn,14JZK1nqHpnZ1imkpv6T5RMGKoCbjg2Q7zZ2omDimFgMvrzk,12GTt3pfM3SjTU6UL6dQ3SMgMSvdw94PnRoF6osU6hPvxbUZ,amount
stash_address,,,,,,,,,,,,,,,,,,,,,
16Ak2D2ZApaR6sFKkawmC8ea93ASD4aB54eTMvEkDjCFZLjp,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,39498643450307
14FzsWSxgWMmXwafTiGf1YdXZgEoAY7dEv2Qr9h1sCHaWFt2,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1084592814654
15LL8mQJQ7wsxrjxMyBxjphkNxciwirC6cQ5AprmmEt8KGNf,0,1,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,36219110590042
11888uchjZxbyc7RRstxZ8qepsskRjJhJatrxryBTdHuWfK,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,2352067003888369
1ZQpD1knimbf4RLUyoipThPdhvEXJHud2sFDSe3cWPVN3q4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3894263543601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16XMi2A7NvUuwt632GH7awa7P5MYJQrteJJ3iB13FCqeXPok,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2780000000000
16A5SmtCw1sTDr6zXdJ9hShqUX1XbYKXG1kk8Qv2BPhTsDNC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,245783340884
14N6uWVnUDZQfNf66HJsSFPLmJHm1p19Y44CBmVoBi9DDbRD,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,14529640
